In [1]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
import sys
sys.path.append('/Users/arthurbabey/Documents/master2/ML/road66/scripts')
from helpers import *
import keras 
import tensorflow as tf


Using TensorFlow backend.


In [2]:
from __future__ import print_function

from keras import regularizers
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten

In [43]:
# Loaded a set of images
root_dir = "../Datasets/training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = len(files) # Use all images
print("Loading " + str(n) + " testing images")
imgs = [load_image(image_dir + files[i]) for i in range(n)]

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " groundtruth")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]

Loading 100 testing images
Loading 100groundtruth


In [53]:
# 
# Preparation training data
# 


# Extract patches from input images
patch_size = 16 # each patch is 16*16 pixels

img_patches = [img_crop(imgs[i], patch_size, patch_size) for i in range(n)]
gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size) for i in range(n)]

# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

img_patches = np.asarray(img_patches)
gt_patches = np.asarray(gt_patches)

x_train = img_patches
y_train = np.asarray([value_to_class(np.mean(gt_patches[i])) for i in range(gt_patches.shape[0])])

print("62500 patches of 16x16 pixels for training are stored in x_train")
print("62500 binary values (1 = road, 0 = bckgrnd) are stored in y_train")

62500 patches of 16x16 pixels for training are stored in x_train
62500 binary values (1 = road, 0 = bckgrnd) are stored in y_train


In [12]:
# 
# Preparation testing data to make a submission
# 

# 
# Ordering image
# 
root_testdir = "../Datasets/test_set_images"
test_names = os.listdir(root_testdir)

num_test = len(test_names)
order = [int(test_names[i].split("_")[1]) for i in range(num_test)]
index = np.argsort(order)

# Load image and reorder them
imgs_test = [load_image(os.path.join(root_testdir, test_names[i], test_names[i]) + ".png") 
             for i in range(num_test)]
imgs_test = [imgs_test[i] for i in index]

# Crop images in patch and Linearize list of patches
patch_size = 16 # each patch is 16*16 pixels

img_patches_test = [img_crop(imgs_test[i], patch_size, patch_size) for i in range(num_test)]


# Linearize list of patches
y_test = np.asarray([img_patches_test[i][j] for i in range(len(img_patches_test)) 
                     for j in range(len(img_patches_test[i]))])

print("72200 patches of 16x16 pixels from the test set are stored in y_test")

72200 patches of 16x16 pixels from the test set are stored in y_test


In [54]:
split_data = False 

#split the x_train and y_train if True

if split_data:

    from sklearn.model_selection import train_test_split

    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.33, random_state=42)
    y_train = np.asarray(y_train)
    y_test = np.asarray(y_test)

In [55]:
#arranging y_train to be used in the CNN 

y_train = np.array(y_train, ndmin = 2)            
y_train = keras.utils.to_categorical(y_train.T, 2)

#y_test = np.array(y_test, ndmin = 2)            
#y_test = keras.utils.to_categorical(y_test.T, 2)

In [15]:
#x_train = tf.keras.utils.normalize(x_train, axis = 1)
#x_test = tf.keras.utils.normalize(x_test, axis = 1)

In [65]:
batch_size = 32
num_classes = 2
epochs = 1
#data_augmentation = True
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'roadNN.h5'

In [66]:
model = Sequential() #create a model

model.add(Conv2D(32, (1, 1), activation = 'relu' , padding='same', #first conv + relu layer
                 data_format='channels_last',
                 input_shape=x_train.shape[1:]))
model.add(Conv2D(32, (1, 1), activation = 'relu' , padding='same')) #conv + relu layer
model.add(MaxPooling2D(pool_size=(2, 2), strides=1, padding='same', data_format=None)) #first pooling layer

model.add(Conv2D(64, (1, 1), activation = 'relu' , padding='same')) #conv + relu layer
model.add(Conv2D(64, (1, 1), activation = 'relu' , padding='same')) #conv + relu layer
model.add(MaxPooling2D(pool_size=(2, 2), strides=1, padding='same', data_format=None)) #pooling layer

model.add(Conv2D(128, (1, 1), activation = 'relu' , padding='same')) #conv + relu layer
model.add(Conv2D(128, (1, 1), activation = 'relu' , padding='same')) #conv + relu layer
model.add(MaxPooling2D(pool_size=(2, 2), strides=1, padding='same', data_format=None)) #pooling layer

model.add(Flatten()) #flatt layer before feeding the fully connected layer
#model.add(Dense(10))
#model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(num_classes, kernel_regularizer=regularizers.l2())) #Fully Connected layer with 2 outputs
model.add(Activation('softmax')) #sigmoid like, to set value between 0 and 1



In [67]:
model.compile(loss='binary_crossentropy', #compile the model
              optimizer='adam',
              metrics=['binary_accuracy'])

model.fit(x_train, y_train, #fitting the model 
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

Train on 41875 samples, validate on 20625 samples
Epoch 1/1
41875/41875 [==============================] - 136s 3ms/step - loss: 0.4922 - binary_accuracy: 0.7606 - val_loss: 0.4707 - val_binary_accuracy: 0.7773


In [57]:
#print the loss and the metrics if using a split_data

scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Test loss: 0.37451173811247856
Test accuracy: 0.8253090977668762


In [33]:
predictions = model.predict(y_test, batch_size=None, verbose=1, 
                            steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False)
predictions

72200/72200 [==============================] - 26s 362us/step


array([[0.7372001 , 0.2627999 ],
       [0.40423077, 0.5957692 ],
       [0.40198064, 0.59801936],
       ...,
       [0.9114895 , 0.08851055],
       [0.97935885, 0.02064116],
       [0.9445132 , 0.05548685]], dtype=float32)

In [34]:
pred = np.zeros(predictions.shape[0]) 
pred[predictions[:,0]< predictions[:,1]] = 1 #merging predictions to 1d vector with value 0 or 1


In [36]:
create_submission(pred, "cnn_submission.csv")


In [59]:
model.save('cnn.model') #to load and save the current model 
#new_model = tf.keras.models.load_model('cnn.model')
